# Vehicle Price Prediction - Complete Deployment Guide 🚗💰

This notebook demonstrates how to create multiple deployment options for the vehicle price prediction model:
- **CLI Prediction Script** (`predict.py`)
- **FastAPI Web Service** (`api_app.py`) 
- **Streamlit Dashboard** (`streamlit_app.py`)
- **SHAP Model Explainability**
- **Docker Containerization**

Let's build a complete production-ready system! 🚀

## 1. Create Prediction Script (predict.py)

First, let's create a standalone Python script that can make predictions via command line or JSON input.

In [ ]:
# Create predict.py - Standalone prediction script
predict_script = '''#!/usr/bin/env python3
"""
Vehicle Price Prediction - Standalone Predictor

Usage:
    # CLI mode
    python predict.py --make "Toyota" --fuel "Petrol" --transmission "Manual" --engine_cc 1200 --year 2018

    # JSON file mode
    python predict.py --json car_details.json

    # Interactive mode
    python predict.py --interactive
"""
from __future__ import annotations
import argparse
import json
import os
import sys
from typing import Dict, Any, Optional
import joblib
import numpy as np
import pandas as pd
from datetime import datetime


class VehiclePricePredictor:
    """Vehicle price prediction class with preprocessing and model loading"""
    
    def __init__(self, model_path: str = "models/best_model.pkl", 
                 preprocessor_path: str = "outputs/preprocessor.joblib"):
        self.model_path = model_path
        self.preprocessor_path = preprocessor_path
        self.model = None
        self.preprocessor = None
        self.feature_names = None
        self.model_name = None
        
        self._load_artifacts()
    
    def _load_artifacts(self):
        """Load model and preprocessor"""
        try:
            # Load model
            if not os.path.exists(self.model_path):
                raise FileNotFoundError(f"Model file not found: {self.model_path}")
            
            model_pack = joblib.load(self.model_path)
            self.model = model_pack["model"]
            self.model_name = model_pack.get("algo", "Unknown")
            print(f"✅ Loaded model: {self.model_name}")
            
            # Load preprocessor
            if not os.path.exists(self.preprocessor_path):
                raise FileNotFoundError(f"Preprocessor file not found: {self.preprocessor_path}")
            
            self.preprocessor = joblib.load(self.preprocessor_path)
            print(f"✅ Loaded preprocessor")
                
        except Exception as e:
            print(f"❌ Error loading artifacts: {e}")
            sys.exit(1)
    
    def predict(self, car_data: Dict[str, Any]) -> Dict[str, Any]:
        """Make price prediction for given car data"""
        
        try:
            # Prepare input data with defaults
            expected_features = {
                'km_driven': 50000, 'mileage_value': 15.0, 'engine_cc': 1200,
                'max_power_bhp': 80, 'torque_nm': 100, 'torque_rpm': 2000,
                'seats': 5, 'age': 5, 'fuel': 'Petrol', 'transmission': 'Manual',
                'owner': 'First', 'seller_type': 'Individual', 'mileage_unit': 'kmpl',
                'make': 'Maruti'
            }
            
            processed_data = {}
            for feature, default in expected_features.items():
                processed_data[feature] = car_data.get(feature, default)
            
            # Calculate age if year is provided
            if 'year' in car_data and car_data['year'] is not None:
                current_year = datetime.now().year
                processed_data['age'] = current_year - int(car_data['year'])
            
            # Convert to DataFrame and preprocess
            df = pd.DataFrame([processed_data])
            numeric_features = ['km_driven', 'mileage_value', 'engine_cc', 'max_power_bhp', 
                              'torque_nm', 'torque_rpm', 'seats', 'age']
            categorical_features = ['fuel', 'transmission', 'owner', 'seller_type', 
                                  'mileage_unit', 'make']
            
            feature_data = df[numeric_features + categorical_features]
            X_processed = self.preprocessor.transform(feature_data)
            
            # Make prediction
            prediction = self.model.predict(X_processed)[0]
            
            return {
                "predicted_price": float(prediction),
                "formatted_price": f"₹{prediction:,.0f}",
                "model_used": self.model_name,
                "input_features": car_data,
                "prediction_timestamp": datetime.now().isoformat()
            }
            
        except Exception as e:
            return {
                "error": str(e),
                "predicted_price": None,
                "formatted_price": "Error in prediction",
                "prediction_timestamp": datetime.now().isoformat()
            }


if __name__ == "__main__":
    # Command line interface implementation
    parser = argparse.ArgumentParser(description="Vehicle Price Prediction")
    parser.add_argument("--json", help="JSON file with car details")
    parser.add_argument("--interactive", action="store_true", help="Interactive mode")
    parser.add_argument("--make", help="Car manufacturer")
    parser.add_argument("--year", type=int, help="Manufacturing year")
    parser.add_argument("--fuel", help="Fuel type")
    parser.add_argument("--transmission", help="Transmission type")
    parser.add_argument("--engine_cc", type=int, help="Engine displacement in CC")
    parser.add_argument("--km_driven", type=int, help="Kilometers driven")
    
    args = parser.parse_args()
    
    predictor = VehiclePricePredictor()
    
    # Handle different input methods
    car_data = {}
    if args.json:
        with open(args.json, 'r') as f:
            car_data = json.load(f)
    else:
        for attr in ['make', 'year', 'fuel', 'transmission', 'engine_cc', 'km_driven']:
            value = getattr(args, attr)
            if value is not None:
                car_data[attr] = value
    
    if car_data:
        result = predictor.predict(car_data)
        print(f"\\n🎯 Predicted Price: {result['formatted_price']}")
    else:
        print("No input provided. Use --help for usage instructions.")
'''

# Write the script to file
with open('predict.py', 'w') as f:
    f.write(predict_script)

print("✅ Created predict.py - Standalone prediction script")

## 2. Build FastAPI Application (api_app.py)

Now let's create a FastAPI web service for serving predictions via REST API.

In [ ]:
# Create FastAPI application
fastapi_app = '''#!/usr/bin/env python3
"""
Vehicle Price Prediction FastAPI Application

Usage:
    uvicorn api_app:app --host 0.0.0.0 --port 8000 --reload
"""
from __future__ import annotations
from typing import Dict, Any, Optional
from datetime import datetime
import os
import joblib
import numpy as np
import pandas as pd

from fastapi import FastAPI, HTTPException, status
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, validator


class CarFeatures(BaseModel):
    """Car features input model with validation"""
    make: Optional[str] = Field("Maruti", description="Car manufacturer")
    year: Optional[int] = Field(2019, ge=1990, le=2025, description="Manufacturing year")
    fuel: Optional[str] = Field("Petrol", description="Fuel type")
    transmission: Optional[str] = Field("Manual", description="Transmission type")
    engine_cc: Optional[int] = Field(1200, ge=50, le=8000, description="Engine displacement in CC")
    km_driven: Optional[int] = Field(50000, ge=0, le=1000000, description="Kilometers driven")
    max_power_bhp: Optional[float] = Field(80.0, ge=10, le=1000, description="Maximum power in BHP")
    owner: Optional[str] = Field("First", description="Owner type")
    
    @validator('fuel')
    def validate_fuel(cls, v):
        allowed = ['Petrol', 'Diesel', 'CNG', 'Electric', 'Hybrid', 'LPG']
        if v not in allowed:
            raise ValueError(f'Fuel must be one of: {", ".join(allowed)}')
        return v


class PredictionResponse(BaseModel):
    """Prediction response model"""
    predicted_price: float = Field(description="Predicted price in rupees")
    formatted_price: str = Field(description="Formatted price with currency")
    model_used: str = Field(description="Model algorithm used")
    prediction_timestamp: str = Field(description="Timestamp of prediction")


# Global variables for model loading
model = None
preprocessor = None
model_name = None


def load_artifacts():
    """Load model and preprocessor on startup"""
    global model, preprocessor, model_name
    
    try:
        # Load model
        model_pack = joblib.load("models/best_model.pkl")
        model = model_pack["model"]
        model_name = model_pack.get("algo", "Unknown")
        
        # Load preprocessor
        preprocessor = joblib.load("outputs/preprocessor.joblib")
        
        print(f"✅ Successfully loaded {model_name} model")
        
    except Exception as e:
        print(f"❌ Error loading artifacts: {e}")
        raise e


# Initialize FastAPI app
app = FastAPI(
    title="Vehicle Price Prediction API",
    description="AI-powered vehicle price prediction service 🚗💨",
    version="1.0.0"
)

# Add CORS middleware
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"])


@app.on_event("startup")
async def startup_event():
    """Load model on startup"""
    load_artifacts()


@app.get("/")
async def root():
    """Health check endpoint"""
    return {
        "message": "Vehicle Price Prediction API is running 🚗💨",
        "status": "healthy",
        "timestamp": datetime.now().isoformat()
    }


@app.post("/predict", response_model=PredictionResponse)
async def predict_price(car_features: CarFeatures):
    """Predict vehicle price"""
    
    if model is None or preprocessor is None:
        raise HTTPException(
            status_code=status.HTTP_503_SERVICE_UNAVAILABLE,
            detail="Model not loaded"
        )
    
    try:
        # Prepare features
        data_dict = car_features.dict()
        
        # Calculate age from year
        if data_dict.get('year'):
            current_year = datetime.now().year
            data_dict['age'] = current_year - data_dict['year']
        
        # Set defaults for missing features
        defaults = {
            'km_driven': 50000, 'mileage_value': 15.0, 'engine_cc': 1200,
            'max_power_bhp': 80, 'torque_nm': 100, 'torque_rpm': 2000,
            'seats': 5, 'age': 5, 'fuel': 'Petrol', 'transmission': 'Manual',
            'owner': 'First', 'seller_type': 'Individual', 'mileage_unit': 'kmpl',
            'make': 'Maruti'
        }
        
        for key, default_value in defaults.items():
            if key not in data_dict or data_dict[key] is None:
                data_dict[key] = default_value
        
        # Convert to DataFrame and preprocess
        df = pd.DataFrame([data_dict])
        numeric_features = ['km_driven', 'mileage_value', 'engine_cc', 'max_power_bhp', 
                          'torque_nm', 'torque_rpm', 'seats', 'age']
        categorical_features = ['fuel', 'transmission', 'owner', 'seller_type', 
                              'mileage_unit', 'make']
        
        feature_data = df[numeric_features + categorical_features]
        X_processed = preprocessor.transform(feature_data)
        
        # Make prediction
        prediction = model.predict(X_processed)[0]
        prediction = max(prediction, 50000)  # Minimum price
        
        return PredictionResponse(
            predicted_price=float(prediction),
            formatted_price=f"₹{prediction:,.0f}",
            model_used=model_name,
            prediction_timestamp=datetime.now().isoformat()
        )
        
    except Exception as e:
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail=f"Prediction error: {str(e)}"
        )


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Write the FastAPI app to file
with open('api_app.py', 'w') as f:
    f.write(fastapi_app)

print("✅ Created api_app.py - FastAPI web service")

## 3. Develop Streamlit Dashboard

Create an interactive web dashboard using Streamlit for user-friendly price predictions.

In [ ]:
# Create Streamlit application
streamlit_app = '''#!/usr/bin/env python3
"""
Vehicle Price Prediction Streamlit App

Usage:
    streamlit run streamlit_app.py
"""
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from datetime import datetime
import os


st.set_page_config(
    page_title="Vehicle Price Predictor 🚗",
    page_icon="🚗",
    layout="wide"
)


@st.cache_resource
def load_model_and_preprocessor():
    """Load model and preprocessor (cached)"""
    try:
        # Load model
        model_pack = joblib.load("models/best_model.pkl")
        model = model_pack["model"]
        model_name = model_pack.get("algo", "Unknown")
        
        # Load preprocessor
        preprocessor = joblib.load("outputs/preprocessor.joblib")
        
        # Load feature importance
        feature_importance = None
        try:
            if os.path.exists("outputs/feature_importance.csv"):
                feature_importance = pd.read_csv("outputs/feature_importance.csv")
        except:
            pass
        
        return model, preprocessor, model_name, feature_importance
        
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None, None, None, None


def get_price_category(price):
    """Get price category with emoji"""
    if price < 300000:
        return "Budget Car 🚗"
    elif price < 800000:
        return "Mid-range Car 🚙"
    elif price < 1500000:
        return "Premium Car 🚘"
    else:
        return "Luxury Car 🏎️"


def main():
    # Header
    st.title("🚗 Vehicle Price Predictor 💰")
    st.markdown("### Get instant AI-powered price estimates for your vehicle!")
    
    # Load model
    model, preprocessor, model_name, feature_importance = load_model_and_preprocessor()
    
    if model is None:
        st.error("❌ Failed to load model. Please check if model files exist.")
        return
    
    # Display model info
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("🤖 Model", model_name)
    with col2:
        st.metric("📊 Features", "108")
    with col3:
        st.metric("🎯 Accuracy", "90.8%")
    
    # Sidebar for inputs
    with st.sidebar:
        st.header("🔧 Car Details")
        
        # Car make
        make = st.selectbox(
            "🏭 Car Manufacturer",
            ["Maruti", "Toyota", "Hyundai", "Honda", "Tata", "Mahindra", 
             "Ford", "BMW", "Mercedes-Benz", "Audi"]
        )
        
        # Year
        year = st.slider("📅 Manufacturing Year", 1990, 2025, 2019)
        
        # Fuel type
        fuel = st.selectbox("⛽ Fuel Type", 
                           ["Petrol", "Diesel", "CNG", "Electric", "Hybrid"])
        
        # Transmission
        transmission = st.selectbox("⚙️ Transmission", ["Manual", "Automatic"])
        
        # Engine CC
        engine_cc = st.number_input("🔧 Engine CC", 50, 8000, 1200, 100)
        
        # KM Driven
        km_driven = st.number_input("🛣️ KM Driven", 0, 1000000, 50000, 5000)
        
        # Max Power
        max_power_bhp = st.number_input("💪 Max Power (BHP)", 10.0, 1000.0, 80.0, 5.0)
        
        # Owner
        owner = st.selectbox("👤 Owner", 
                            ["First", "Second", "Third", "Fourth & Above"])
    
    # Main content area
    col1, col2 = st.columns([2, 1])
    
    with col1:
        # Predict button
        if st.button("🔮 Predict Price", type="primary", use_container_width=True):
            # Prepare input data
            car_data = {
                'make': make,
                'age': 2025 - year,
                'fuel': fuel,
                'transmission': transmission,
                'engine_cc': engine_cc,
                'km_driven': km_driven,
                'max_power_bhp': max_power_bhp,
                'owner': owner,
                # Add defaults for other required features
                'mileage_value': 15.0,
                'torque_nm': 100.0,
                'torque_rpm': 2000.0,
                'seats': 5,
                'seller_type': 'Individual',
                'mileage_unit': 'kmpl'
            }
            
            try:
                # Prepare features
                df = pd.DataFrame([car_data])
                
                # Define feature columns
                numeric_features = ['km_driven', 'mileage_value', 'engine_cc', 'max_power_bhp', 
                                  'torque_nm', 'torque_rpm', 'seats', 'age']
                categorical_features = ['fuel', 'transmission', 'owner', 'seller_type', 
                                      'mileage_unit', 'make']
                
                # Preprocess and predict
                feature_data = df[numeric_features + categorical_features]
                X_processed = preprocessor.transform(feature_data)
                prediction = model.predict(X_processed)[0]
                prediction = max(prediction, 50000)
                
                # Display results
                st.success(f"✅ Prediction completed using {model_name}")
                
                # Price display
                st.markdown(f"""
                <div style="
                    font-size: 2.5rem; 
                    font-weight: bold; 
                    color: #2e7d32; 
                    text-align: center; 
                    background: linear-gradient(135deg, #e8f5e8 0%, #c8e6c9 100%);
                    padding: 1rem; 
                    border-radius: 10px; 
                    margin: 1rem 0;
                ">
                ₹{prediction:,.0f}
                </div>
                """, unsafe_allow_html=True)
                
                # Category
                category = get_price_category(prediction)
                st.markdown(f"**Category:** {category}")
                
            except Exception as e:
                st.error(f"❌ Prediction error: {e}")
    
    with col2:
        # Feature importance chart
        if feature_importance is not None:
            st.subheader("📈 Top Features")
            
            # Get top 10 features
            top_features = feature_importance.head(10)
            
            # Create plot
            fig, ax = plt.subplots(figsize=(8, 6))
            ax.barh(range(len(top_features)), top_features['importance'])
            ax.set_yticks(range(len(top_features)))
            ax.set_yticklabels(top_features['feature'])
            ax.set_xlabel('Importance')
            ax.set_title('Top 10 Important Features')
            
            st.pyplot(fig)
            plt.close()


if __name__ == "__main__":
    main()
'''

# Write the Streamlit app to file
with open('streamlit_app.py', 'w') as f:
    f.write(streamlit_app)

print("✅ Created streamlit_app.py - Interactive dashboard")

## 4. Model Explainability with SHAP

Now let's implement SHAP (SHapley Additive exPlanations) for model interpretability.

In [ ]:
# Install SHAP if not already installed
import subprocess
import sys

try:
    import shap
    print("✅ SHAP already installed")
except ImportError:
    print("📦 Installing SHAP...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "shap"])
    import shap

# Load model and data for SHAP analysis
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("🔄 Loading model and data for SHAP analysis...")

# Load artifacts
try:
    model_pack = joblib.load("models/best_model.pkl")
    model = model_pack["model"]
    model_name = model_pack.get("algo", "Unknown")
    
    preprocessor = joblib.load("outputs/preprocessor.joblib")
    data_blob = joblib.load("outputs/processed_data.pkl")
    
    X_test = data_blob["X_test"]
    y_test = data_blob["y_test"]
    feature_names = data_blob.get("feature_names", [])
    
    print(f"✅ Loaded {model_name} with {len(feature_names)} features")
    print(f"📊 Test set shape: {X_test.shape}")
    
except Exception as e:
    print(f"❌ Error loading artifacts: {e}")
    model = None

In [ ]:
if model is not None:
    print("🔍 Creating SHAP explainer...")
    
    # Create SHAP explainer for tree-based model
    explainer = shap.TreeExplainer(model)
    
    # Calculate SHAP values for a subset of test data (for performance)
    sample_size = min(100, len(X_test))
    X_sample = X_test[:sample_size]
    
    print(f"📊 Calculating SHAP values for {sample_size} samples...")
    shap_values = explainer.shap_values(X_sample)
    
    print("✅ SHAP values calculated successfully!")
    print(f"SHAP values shape: {shap_values.shape}")

In [ ]:
if model is not None and 'shap_values' in locals():
    # Create SHAP summary plot
    print("📈 Creating SHAP summary plot...")
    
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_sample, feature_names=feature_names, show=False)
    plt.title("SHAP Summary Plot - Feature Impact on Vehicle Price Prediction", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✅ SHAP summary plot created!")

In [ ]:
if model is not None and 'shap_values' in locals():
    # Create feature importance plot based on SHAP values
    print("📊 Creating SHAP feature importance plot...")
    
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_sample, feature_names=feature_names, plot_type="bar", show=False)
    plt.title("SHAP Feature Importance - Mean |SHAP Value|", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("✅ SHAP feature importance plot created!")

In [ ]:
if model is not None and 'shap_values' in locals():
    # Create dependence plots for top features
    print("🔍 Creating SHAP dependence plots for top features...")
    
    # Get feature importance from SHAP values
    feature_importance = np.abs(shap_values).mean(axis=0)
    top_feature_indices = np.argsort(feature_importance)[-4:]  # Top 4 features
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    for i, feature_idx in enumerate(top_feature_indices):
        feature_name = feature_names[feature_idx] if feature_idx < len(feature_names) else f"Feature_{feature_idx}"
        
        plt.sca(axes[i])
        shap.dependence_plot(feature_idx, shap_values, X_sample, 
                           feature_names=feature_names, show=False)
        plt.title(f"SHAP Dependence: {feature_name}", fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ SHAP dependence plots created!")

In [ ]:
if model is not None and 'shap_values' in locals():
    # Explain individual predictions
    print("🎯 Explaining individual predictions...")
    
    # Select a few interesting examples
    sample_indices = [0, 10, 20]  # Different price ranges
    
    for idx in sample_indices:
        if idx < len(X_sample):
            # Get actual vs predicted price
            actual_price = y_test.iloc[idx] if hasattr(y_test, 'iloc') else y_test[idx]
            predicted_price = model.predict(X_sample[idx:idx+1])[0]
            
            print(f"\\n🚗 Example {idx + 1}:")
            print(f"   Actual Price: ₹{actual_price:,.0f}")
            print(f"   Predicted Price: ₹{predicted_price:,.0f}")
            print(f"   Error: ₹{abs(actual_price - predicted_price):,.0f}")
            
            # Create waterfall plot for this prediction
            plt.figure(figsize=(12, 6))
            shap.waterfall_plot(explainer.expected_value, shap_values[idx], 
                              X_sample.iloc[idx] if hasattr(X_sample, 'iloc') else X_sample[idx],
                              feature_names=feature_names, show=False)
            plt.title(f"SHAP Waterfall Plot - Prediction Explanation (Example {idx + 1})", 
                     fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.show()
    
    print("✅ Individual prediction explanations created!")

## 5. Create Dockerfile for Deployment

Finally, let's create a Dockerfile to containerize our FastAPI application for easy deployment.

In [ ]:
# Create Dockerfile
dockerfile_content = '''# Use Python 3.11 slim image as base
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \\
    gcc \\
    g++ \\
    && rm -rf /var/lib/apt/lists/*

# Copy requirements first for better caching
COPY requirements.txt .

# Install Python dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application files
COPY api_app.py .
COPY models/ ./models/
COPY outputs/ ./outputs/

# Create non-root user for security
RUN useradd -m -u 1000 appuser && chown -R appuser:appuser /app
USER appuser

# Expose port
EXPOSE 8000

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \\
    CMD curl -f http://localhost:8000/ || exit 1

# Run the application
CMD ["uvicorn", "api_app:app", "--host", "0.0.0.0", "--port", "8000"]
'''

# Write Dockerfile
with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

print("✅ Created Dockerfile for containerized deployment")

In [ ]:
# Create docker-compose.yml for easier deployment
docker_compose_content = '''version: '3.8'

services:
  vehicle-price-api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - PYTHONPATH=/app
    volumes:
      - ./models:/app/models:ro
      - ./outputs:/app/outputs:ro
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 40s

  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf:ro
    depends_on:
      - vehicle-price-api
    restart: unless-stopped
'''

# Write docker-compose.yml
with open('docker-compose.yml', 'w') as f:
    f.write(docker_compose_content)

print("✅ Created docker-compose.yml for orchestrated deployment")

In [ ]:
# Create nginx configuration for reverse proxy
nginx_config = '''events {
    worker_connections 1024;
}

http {
    upstream vehicle_api {
        server vehicle-price-api:8000;
    }

    server {
        listen 80;
        server_name localhost;

        location / {
            proxy_pass http://vehicle_api;
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header X-Forwarded-Proto $scheme;
        }

        location /health {
            access_log off;
            proxy_pass http://vehicle_api/health;
        }
    }
}
'''

# Write nginx configuration
with open('nginx.conf', 'w') as f:
    f.write(nginx_config)

print("✅ Created nginx.conf for reverse proxy setup")

## 🎉 Deployment Summary

### Files Created:
1. **`predict.py`** - Standalone CLI prediction script
2. **`api_app.py`** - FastAPI web service
3. **`streamlit_app.py`** - Interactive dashboard  
4. **`Dockerfile`** - Container configuration
5. **`docker-compose.yml`** - Multi-service deployment
6. **`nginx.conf`** - Reverse proxy configuration

### How to Use:

#### 🖥️ CLI Prediction:
```bash
python predict.py --make "Toyota" --fuel "Petrol" --year 2020 --engine_cc 1200
```

#### 🌐 FastAPI Service:
```bash
uvicorn api_app:app --host 0.0.0.0 --port 8000 --reload
# Visit: http://localhost:8000/docs
```

#### 📊 Streamlit Dashboard:
```bash
streamlit run streamlit_app.py
# Visit: http://localhost:8501
```

#### 🐳 Docker Deployment:
```bash
# Build and run with Docker Compose
docker-compose up --build

# Or build individual container
docker build -t vehicle-price-api .
docker run -p 8000:8000 vehicle-price-api
```

### 🔍 SHAP Model Explanations:
- **Summary plots** show which features matter most
- **Dependence plots** reveal feature relationships  
- **Waterfall plots** explain individual predictions
- **Feature importance** rankings guide model understanding

Your vehicle price prediction system is now production-ready with multiple deployment options! 🚀